Bayan Nezamabad - 20251971, Euan Bourke - 21332142

In [49]:
# Imports
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
import matplotlib.pyplot as plt
from keras import utils

In [60]:
# Hyper Params
EPOCHS = 20
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = optimizers.Adam()
VALIDATION_SPLIT=0.90

IMG_ROW, IMG_COL = 224, 224 # Image dimensions
INPUT_SHAPE = (IMG_ROW, IMG_COL, 3) # 3 as RGB
NB_CLASSES = 100 # 100 different classifications for 100 different species of butterfly or moth

TRAIN_PATH = 'data/train'
VALID_PATH = 'data/valid'
SEED = 123 # used for randomising dataset (to keep it predictable)

In [66]:
class Dataset:
    def __init__(self):
        self.training_data = utils.image_dataset_from_directory(TRAIN_PATH,
                                                                shuffle=True,
                                                                image_size=(IMG_COL, IMG_ROW),
                                                                batch_size=BATCH_SIZE,
                                                                seed=SEED)
        self.valid_data = utils.image_dataset_from_directory(VALID_PATH,
                                                             shuffle=True,
                                                             image_size=(IMG_COL, IMG_ROW),
                                                             batch_size=BATCH_SIZE,
                                                             seed=SEED)
    
    def load_training_data(self):
        return self.training_data
    
    def load_valid_data(self):
        return self.valid_data
    

data = Dataset()
train_data = data.load_training_data()
valid_data = data.load_valid_data()

Found 12594 files belonging to 100 classes.
Found 500 files belonging to 100 classes.


In [67]:
class BENet: # Our implementation of AlexNet
    def __init__(self):
        self.model = tf.keras.models.Sequential()
        self.build_layers()
        self.model.compile(optimizer=optimizers.SGD(learning_rate = 0.001), loss="sparse_categorical_crossentropy", metrics=['accuracy'])

    def build_layers(self):

        # Convolutional Layers
        self.model.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=INPUT_SHAPE))
        self.model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        self.model.add(layers.BatchNormalization())

        self.model.add(layers.Conv2D(256, (5, 5), padding='same', activation='relu'))
        self.model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        self.model.add(layers.BatchNormalization())

        self.model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
        self.model.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
        self.model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))

        self.model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

        self.model.add(layers.Flatten()) # Flatten Layers

        # Full Connected Layers
        self.model.add(layers.Dropout(rate=0.5))
        self.model.add(layers.Dense(4096, activation='relu'))

        self.model.add(layers.Dropout(rate=0.5))
        self.model.add(layers.Dense(4096, activation='relu'))
        
        self.model.add(layers.Dense(NB_CLASSES, activation='softmax'))
        
        return self.model

model = BENet()
model.model.summary()

/Users/euanbourke/Documents/cs4287-assignment1/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_44 (Conv2D)              │ (None, 54, 54, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 26, 26, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 26, 26, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_45 (Conv2D)              │ (None, 26, 26, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 12, 12, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 12, 12, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 12, 12, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 4096)           │    26,218,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 100)            │       409,700 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,158,116 (179.89 MB)

 Trainable params: 47,157,412 (179.89 MB)

 Non-trainable params: 704 (2.75 KB)

In [68]:
model_summary = model.model.fit(train_data, epochs=EPOCHS, validation_data=valid_data)

Epoch 1/20


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 100)

Refs: \
AlexNet Architecture: https://www.kaggle.com/code/blurredmachine/alexnet-architecture-a-complete-guide \
https://medium.com/@siddheshb008/alexnet-architecture-explained-b6240c528bd5
Keras.layer docs: https://www.tensorflow.org/api_docs/python/tf/keras/layers/
